<a href="https://colab.research.google.com/github/epireve/bg.rm/blob/master/Remove_bg_CNN_Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/eti-p-doray/unet-gan-matting

Cloning into 'unet-gan-matting'...
remote: Enumerating objects: 319, done.
remote: Total 319 (delta 0), reused 0 (delta 0), pack-reused 319
Receiving objects: 100% (319/319), 141.09 MiB | 40.56 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [0]:
cd unet-gan-matting/

/content/unet-gan-matting


In [0]:
ls

eval.py                          LICENSE    requirements-gpu.txt  train.py
exploringImageMattingReport.pdf  log/       requirements.txt      unet/
images/                          README.md  scripts/


In [0]:
!pip3 install -r requirements-gpu.txt

     |████████████████████████████████| 2.0MB 2.8MB/s 
     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 911kB 34.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/9a/98/65/c1313c5ed4a9d5a6c6620cc92b167d8b637c3a9d05a2511bb9
Successfully built google-images-download
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 4.3.0
    Uninstalling Pillow-4.3.0:
      Successfully uninstalled Pillow-4.3.0


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!python3 unet-gan-matting/scripts/download.py --chromedriver


Item no.: 1 --> Item name = portrait transparent background
Evaluating...
Looks like we cannot locate the path the 'chromedriver' (use the '--chromedriver' argument to specify the path to the executable.) or google chrome browser is not installed on your machine (exception: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
)


In [0]:
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get("https://www.says.com")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (73.0.3683.86-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [0]:
!python3 unet-gan-matting/scripts/download.py --chromedriver "/usr/bin"


Item no.: 1 --> Item name = portrait transparent background
Evaluating...
Looks like we cannot locate the path the 'chromedriver' (use the '--chromedriver' argument to specify the path to the executable.) or google chrome browser is not installed on your machine (exception: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
)


In [0]:
!ls /usr/bin/chromedriver

In [0]:
import os
import shutil

from google_images_download import google_images_download

def dowload_matting_dataset(output_dir):

    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)

    response = google_images_download.googleimagesdownload()
    response.download({
        "keywords": "portrait transparent background",
        "color_type": "transparent",
        "size": "medium",
        "limit": 500,
        "output_directory": output_dir,
        "chromedriver": "/usr/bin/chromedriver"})

    response = google_images_download.googleimagesdownload()
    response.download({
        "keywords": "texture background",
        "color_type": "full-color",
        "size": "medium",
        "limit": 500,
        "output_directory": output_dir,
        "chromedriver": "/usr/bin/chromedriver"})

if __name__ == "__main__":
    output_dir = os.path.join("data", "matting")
    dowload_matting_dataset(output_dir)


Item no.: 1 --> Item name = portrait transparent background
Evaluating...
Getting you a lot of images. This may take a few moments...
Reached end of Page.
Starting Download...
Completed Image ====> 1. 5866204a7d90850fc3ce2a72.png
Completed Image ====> 2. 5874e05742e4d628738559fa.png
Completed Image ====> 3. 145519_thumb.png
Completed Image ====> 4. 5890d7fab2cb5b18709ca61a.png
Completed Image ====> 5. Chris-Pine-Transparent-Background.png
Completed Image ====> 6. 584be7749348a282db9585c9.png
Completed Image ====> 7. woman-1781508_960_720.png
Completed Image ====> 8. canva-portrait-of-smiling-young-man-isolated-on-transparent-background-MACFVd9Or3Y.png
Completed Image ====> 9. portrait-femme-11551057725piimbdmzkf.png
Completed Image ====> 10. 35235-5-margot-robbie-transparent-background.png
Completed Image ====> 11. 59300abe3919fe0ee3614dc6.png
Completed Image ====> 12. canva-portrait-of-male-engineer-looking-sideways%2C-isolated-on-transparent-background-MAB7Y_5m0YE.png
Completed Imag

In [0]:
!ls /content/data/matting/input

In [0]:
import os
import cv2
import random
import numpy
import math


def image_fill(img, size, value):

    border = [math.ceil((size[0] - img.shape[0])/2),
              math.floor((size[0] - img.shape[0])/2),
              math.ceil((size[1] - img.shape[1])/2),
              math.floor((size[1] - img.shape[1])/2)]
    return cv2.copyMakeBorder(img,border[0],border[1],border[2],border[3],cv2.BORDER_CONSTANT,value=value)


def combine_object_background(object_file, background_file, output_name):
    border = 20
    size = [960, 720]

    foreground = cv2.imread(object_file, cv2.IMREAD_UNCHANGED)
    if foreground is None:
        return False

    ratio = numpy.amin(numpy.divide(
            numpy.subtract(size, [2*border, 2*border]), foreground.shape[0:2]))
    forground_size = numpy.floor(numpy.multiply(foreground.shape[0:2], ratio)).astype(int)
    foreground = cv2.resize(foreground, (forground_size[1], forground_size[0]))
    foreground = image_fill(foreground,size,[0,0,0,0])

    foreground = foreground.astype(float)
    cv2.normalize(foreground, foreground, 0.0, 1.0, cv2.NORM_MINMAX)
    alpha = cv2.split(foreground)[3]

    #foreground = cv2.imread(object_file, cv2.IMREAD_COLOR)
    background = cv2.imread(background_file)
    if background is None:
        return False

    ratio = numpy.amax(numpy.divide(foreground.shape[0:2], background.shape[0:2]))
    background_size = numpy.ceil(numpy.multiply(background.shape[0:2], ratio)).astype(int)
    #print(numpy.multiply(background.shape[0:2], ratio).astype(int))
    background = cv2.resize(background, (background_size[1], background_size[0]))
    background = background[0:foreground.shape[0], 0:foreground.shape[1]]
    background = background.astype(float)

    for i in range(0, 3):
        foreground[:,:,i] = numpy.multiply(alpha, foreground[:,:,i]*255)
        background[:,:,i] = numpy.multiply(1.0 - alpha, background[:,:,i])
    outImage = numpy.add(foreground[:,:,0:3], background)

    cv2.imwrite(output_name, outImage)

    return True


def generate_trimap(object_file, trimap_name):
    border = 20
    size = [960, 720]

    foreground = cv2.imread(object_file, cv2.IMREAD_UNCHANGED)
    if foreground is None:
        return False
    alpha = cv2.split(foreground)[3]

    ratio = numpy.amin(numpy.divide(
            numpy.subtract(size, [2*border, 2*border]), alpha.shape[0:2]))
    forground_size = numpy.floor(numpy.multiply(alpha.shape[0:2], ratio)).astype(int)
    alpha = cv2.resize(alpha, (forground_size[1], forground_size[0]))
    alpha = image_fill(alpha,size,[0,0,0,0])

    alpha = alpha.astype(float)
    cv2.normalize(alpha, alpha, 0.0, 1.0, cv2.NORM_MINMAX)

    _, inner_map = cv2.threshold(alpha, 0.999, 255, cv2.THRESH_BINARY)
    _, outer_map = cv2.threshold(alpha, 0.001, 255, cv2.THRESH_BINARY)

    inner_map = cv2.erode(inner_map, numpy.ones((5,5),numpy.uint8), iterations = 3)
    outer_map = cv2.dilate(outer_map, numpy.ones((5,5),numpy.uint8), iterations = 3)

    cv2.imwrite(trimap_name, inner_map + (outer_map - inner_map) /2)

    foreground = cv2.imread(object_file, cv2.IMREAD_UNCHANGED)

def generate_target(object_file, target_name):
    border = 20
    size = [960, 720]

    foreground = cv2.imread(object_file, cv2.IMREAD_UNCHANGED)
    if foreground is None:
        return False
    cv2.normalize(foreground, foreground, 0, 255, cv2.NORM_MINMAX)
    foreground = foreground.astype(numpy.uint8)

    ratio = numpy.amin(numpy.divide(
            numpy.subtract(size, [2*border, 2*border]), foreground.shape[0:2]))
    forground_size = numpy.floor(numpy.multiply(foreground.shape[0:2], ratio)).astype(int)
    foreground = cv2.resize(foreground, (forground_size[1], forground_size[0]))
    foreground = image_fill(foreground,size,[0,0,0,0])

    cv2.imwrite(target_name, foreground)


def build_dataset(object_dir, background_dir, input_dir, trimap_dir, target_dir):
    object_filenames = os.listdir(object_dir)
    background_filenames = os.listdir(background_dir)

    for i, object_file in enumerate(object_filenames):
        generate_trimap(
            os.path.join(object_dir, object_file),
            os.path.join(trimap_dir, str(i) + '_trimap.jpg'))
        generate_target(
            os.path.join(object_dir, object_file),
            os.path.join(target_dir, str(i) + '.png'))

        backgrounds = random.sample(background_filenames, 20)
        for j, background_file in enumerate(backgrounds):
            print(i, j, object_file, background_file)
            combine_object_background(os.path.join(object_dir, object_file),
              os.path.join(background_dir, background_file),
              os.path.join(input_dir, str(i) + '_' + str(j) + '.jpg'))


if __name__ == "__main__":
    object_dir = os.path.join("data", "matting", "portrait transparent background")
    background_dir = os.path.join("data", "matting", "texture background")
    input_dir = os.path.join("data", "matting", "input")
    trimap_dir = os.path.join("data", "matting", "trimap")
    target_dir = os.path.join("data", "matting", "target")

    if not os.path.isdir(input_dir):
        os.makedirs(input_dir)
    if not os.path.isdir(trimap_dir):
        os.makedirs(trimap_dir)
    if not os.path.isdir(target_dir):
        os.makedirs(target_dir)

    build_dataset(object_dir, background_dir, input_dir, trimap_dir, target_dir)

0 0 80. justin-timberlake-clipart-transparent-background-655251-1463991.png 27. turquoise-textured-background-1488752042N3o.jpg
0 1 80. justin-timberlake-clipart-transparent-background-655251-1463991.png 564. wood-texture-background-vector.jpg
0 2 80. justin-timberlake-clipart-transparent-background-655251-1463991.png 80. golden-marble-texture-background-for-card-vector-21895884.jpg
0 3 80. justin-timberlake-clipart-transparent-background-655251-1463991.png 217. 500_F_178488082_ANh5WM6PX2lLEKQ2QXzPEocLei0dJoYJ.jpg
0 4 80. justin-timberlake-clipart-transparent-background-655251-1463991.png 408. d0136f768bf1f7a0712ac8742edf01cb.jpg
0 5 80. justin-timberlake-clipart-transparent-background-655251-1463991.png 520. 74714067-cherry-wood-texture-background-on-macro-extremely-high-resolution-photo-.jpg
0 6 80. justin-timberlake-clipart-transparent-background-655251-1463991.png 24. geometric-soft-blue-texture-background-wide-screen-vector-21866941.jpg
0 7 80. justin-timberlake-clipart-transparen

In [0]:
!ls

adc.json  data	sample_data  unet-gan-matting


In [0]:
!python3 unet-gan-matting/train.py data/matting

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.conv2d instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dense instead.
2019-05-06 0

In [0]:
!ls data/matting

 input				    target		  trimap
'portrait transparent background'  'texture background'


##Save Model and data into Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
model_save_name = 'classifier.pt'
path = F"/content/gdrive/My Drive/TF Model/Remove.bground model/{model_save_name}" 
torch.save(model.state_dict(), path)

In [0]:
cp -r data /content/gdrive/My\ Drive/TF\ Model/Remove.bground\ model\

In [0]:
cp -r log /content/gdrive/My\ Drive/TF\ Model/Remove.bground\ model\

In [0]:
!rm -r /content/gdrive/My\ Drive/TF\ Model/Remove.bground\ model/matting

In [0]:
!ls -L /content/gdrive/My\ Drive/TF\ Model/Remove.bground\ model

data  log


In [0]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = '[your Cloud Platform project ID]'
!gcloud config set project {project_id}

In [0]:
# Create a local file with data to upload.
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

import uuid

# Make a unique bucket to which we'll upload the file.
# (GCS buckets are part of a single global namespace.)
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
!gsutil mb gs://{bucket_name}

# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/
  
# Finally, dump the contents of our newly copied file to make sure everything worked.
!gsutil cat gs://{bucket_name}/to_upload.txt

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a text file.
uploaded = drive.CreateFile({'title': 'Sample file.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1neZq_eBfkA5R2rmUJA97AnEGxWahQ6qv
